# Fogo DOOM

In [3]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io
import time

# --- 1. Configurações ---
LARGURA, ALTURA = 100, 100
TOTAL_FRAMES = 90  # 3 segundos de fogo pré-calculado
FPS_PLAYBACK = 30

# Paleta do Doom (RGB)
CORES = [
    (7, 7, 7), (31, 7, 7), (47, 15, 7), (71, 15, 7), (87, 23, 7), (103, 31, 7), (119, 31, 7), (143, 39, 7),
    (159, 47, 7), (175, 63, 7), (191, 71, 7), (199, 71, 7), (223, 79, 7), (223, 87, 7), (223, 87, 7), (215, 95, 7),
    (215, 95, 7), (215, 103, 15), (207, 111, 15), (207, 119, 15), (207, 127, 15), (207, 135, 23), (199, 135, 23), (199, 143, 23),
    (199, 151, 31), (191, 159, 31), (191, 159, 31), (191, 167, 39), (191, 167, 39), (191, 175, 47), (183, 175, 47), (183, 183, 47),
    (183, 183, 55), (207, 207, 111), (223, 223, 159), (239, 239, 199), (255, 255, 255)
]
PALETA = np.array(CORES + [(255, 255, 255)], dtype=np.uint8)

# --- 2. O Motor de Processamento (Geração do Cubo de Dados) ---
def gerar_fogo_vetorizado():
    print(" Processando Cubo de Dados (Space-Time Cube)...")
    start_time = time.time()
    
    # Alocação de memória para o Cubo 3D (Tempo, Y, X)
    fogo_cubo = np.zeros((TOTAL_FRAMES, ALTURA, LARGURA), dtype=np.int32)
    fogo_cubo[:, -1, :] = 36  # Base sempre quente (Broadcast temporal)
    
    # Pré-gera aleatoriedade (Vetorização total da aleatoriedade)
    decay_tensor = np.random.randint(0, 3, (TOTAL_FRAMES, ALTURA, LARGURA))
    shifts_horizontal = np.random.randint(-1, 2, (TOTAL_FRAMES, ALTURA))
    
    # Loop Temporal (Inevitável devido à causalidade: t depende de t-1)
    for t in range(1, TOTAL_FRAMES):
        frame_anterior = fogo_cubo[t-1].copy()
        
        # Propagação vertical com decay
        calor_propagado = np.maximum(0, frame_anterior[1:, :] - decay_tensor[t, 1:, :])
        frame_anterior[:-1, :] = calor_propagado
        
        # Vento constante (Roll vetorizado em X)
        frame_anterior[:-1, :] = np.roll(frame_anterior[:-1, :], -1, axis=1)
        
        # Turbulência horizontal (Iteração linha a linha - Gargalo mantido do original)
        for y in range(ALTURA - 1):
            frame_anterior[y, :] = np.roll(frame_anterior[y, :], shifts_horizontal[t, y])
        
        frame_anterior[-1, :] = 36 # Reforça a base
        fogo_cubo[t] = frame_anterior
    
    print(f" Processamento concluído em {time.time() - start_time:.4f}s. Shape: {fogo_cubo.shape}")
    return fogo_cubo

# --- 3. Player de Visualização (Jupyter Widget) ---
def reproduzir_cubo(cubo_dados):
    # Converte TODO o cubo de índices para RGB de uma vez (Broadcasting massivo)
    # Resultado: Tensor 4D (Frames, Altura, Largura, 3 canais RGB)
    print(" Mapeando Paleta de Cores...")
    cubo_rgb = PALETA[cubo_dados] 
    
    img_widget = widgets.Image(
        format='png', width=800, height=600,
        layout=widgets.Layout(object_fit='contain')
    )
    btn_replay = widgets.Button(description="Replay 🔄")
    lbl_status = widgets.Label(value="Status: Pronto")
    
    display(widgets.VBox([lbl_status, img_widget, btn_replay]))
    
    def play(_=None):
        lbl_status.value = "Status: Reproduzindo ▶️"
        btn_replay.disabled = True
        
        for t in range(TOTAL_FRAMES):
            # Extrai frame t
            frame_rgb = cubo_rgb[t]
            
            # Renderiza
            img = Image.fromarray(frame_rgb, 'RGB')
            img = img.resize((800, 600), resample=Image.NEAREST)
            
            with io.BytesIO() as buffer:
                img.save(buffer, format='PNG')
                img_widget.value = buffer.getvalue()
            
            # Controle de FPS
            time.sleep(1.0 / FPS_PLAYBACK)
            
        lbl_status.value = "Status: Fim do Loop ⏹️"
        btn_replay.disabled = False

    btn_replay.on_click(play)
    play() # Auto-start

# Execução
dados_fogo = gerar_fogo_vetorizado()
reproduzir_cubo(dados_fogo)

 Processando Cubo de Dados (Space-Time Cube)...
 Processamento concluído em 0.1183s. Shape: (90, 100, 100)
 Mapeando Paleta de Cores...


C:\Users\pedro\AppData\Local\Temp\ipykernel_22876\2908620357.py:82: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(frame_rgb, 'RGB')
